In [5]:
#!usr/bin/env python3
#
# MY, 14 Nov 2022
# mmy@stanford.edu

"""
U-net testing
  
  This script was written for trying out U-nets.

  Note:
    tensorflow package is installed via tensorflow-macos
    ipywidget was installed additionally
    `Concatenate` is used to replace `merge` from keras.layers
    `image_dim_ordering() == 'th'` --> `image_data_format()=='channels_first`

  Maybe later?
  https://medium.com/analytics-vidhya/creating-a-very-simple-u-net-model-with-pytorch-for-semantic-segmentation-of-satellite-images-223aa216e705

"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, Convolution2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Dropout
from keras.layers.core import Lambda

from keras.models import load_model
import sys
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt

import keras
keras.backend.image_data_format() =='channels_first'
# Theano dimension ordering in this code
import cv2

from unetmodels import multi_unet_model


In [18]:
# Simple UNet

def simple_unet_model(n_classes, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):

    """
    First UNet, writing for practice and understanding. 

    """

    # Since you set "channels_first"
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))


    # Contracting path
    c1 = Conv2D(
        filters=16, 
        kernel_size=(3, 3), 
        activation='relu',
        kernel_initializer='he_normal',
        padding='same'
        )(inputs)
    c1 = Dropout(0.1)(c1)  # To prevent overfitting
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)  # To prevent overfitting
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.1)(c3)  # To prevent overfitting
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)

    # Expanding path
    u4 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u4)
    c4 = Dropout(0.1)(c4)
    c4 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    
    u5 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u5)
    c5 = Dropout(0.1)(c5)
    c5 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    outputs = Conv2D(4, (1,1), activation='softmax')(c5)

    model = Model(inputs=[inputs], outputs=[outputs])

    return model


IMG_HEIGHT      = 496
IMG_WIDTH       = 768
IMG_CHANNELS    = 1


model = simple_unet_model(2, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

In [ ]:
# Training images?

# UNet from UW 2017, Aaron Lee

Link to code: https://github.com/uw-biomedical-ml/irf-segmenter/blob/master/run.py


In [2]:
# Parameters

weightsfile = 'weights.hdf5'    # supplied in github.
image_rows = 768
image_cols = 496
image_channels = 1

def UNET():

    """
    Aaron Lee UW code 
    Code found here: https://github.com/uw-biomedical-ml/irf-segmenter/blob/master/run.py
    
    """

    inputs = Input((1, image_rows, image_cols))
    conv1 = Convolution2D(32, 3, 3, activation="relu", border_mode="same")(inputs)  # n_filters, nrow, ncol of conv kernel
    conv1 = Convolution2D(32, 3, 3, activation="relu", border_mode="same")(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))
